In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_hub as hub



#                  Pretrained-Model                

In [8]:
from google.colab import drive 
import random

random.seed(42)
gdrive_path = '/content/gdrive'
drive.mount(gdrive_path)

Mounted at /content/gdrive


In [9]:
dataset_path = '/content/gdrive/My Drive/Colab Notebooks/CV/Dataset_Transferlearning/'

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

model = keras.models.load_model(dataset_path+"pretrained")

# Freeze all model layer weights
model.trainable = False

# Can also set trainable for specific layers
for layer in model.layers:
    # assert should be true because of one-liner above
    assert layer.trainable == False
    layer.trainable = False

print(model.summary())  # for finding base input and output
base_inputs = model.layers[0].input
base_output = model.layers[-2].output
output = layers.Dense(10)(base_output)
new_model = keras.Model(base_inputs, output)

# This model is actually identical to model we
# loaded (this is just for demonstration and
# and not something you would do in practice).
print(new_model.summary())

# As usual we do compile and fit, this time on new_model
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

new_model.fit(x_train, y_train, batch_size=32, epochs=3, verbose=2)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 32)        4640      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 6272)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                62730     
Total params: 67,530
Trainable params: 0
Non-trainable params: 67,530
_________________________________________________________________
None
Model: "model_1"
_________________________________________________________________
Layer (type)               

#                Pretrained Keras Model 

In [ ]:
# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

model = keras.applications.InceptionV3(include_top=True)
print(model.summary())

# for input you can also do model.input,
# then for base_outputs you can obviously
# choose other than simply removing the last one :)
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
classifier = layers.Dense(3)(base_outputs)
new_model = keras.Model(inputs=base_inputs, outputs=classifier)
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

print(new_model.summary())
new_model.fit(x, y, epochs=15, verbose=2)

96116736/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________

#                Pretrained Hub Model   

In [ ]:
# Random data for demonstration (3 examples w. 3 classes)
x = tf.random.normal(shape=(3, 299, 299, 3))
y = tf.constant([0, 1, 2])

url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4"

base_model = hub.KerasLayer(url, input_shape=(299, 299, 3))
model = keras.Sequential(
    [
        base_model,
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x, y, batch_size=32, epochs=15, verbose=2)

Epoch 1/15
1/1 - 6s - loss: 1.9190 - accuracy: 0.3333
Epoch 2/15
1/1 - 1s - loss: 1.3600 - accuracy: 0.3333
Epoch 3/15
1/1 - 1s - loss: 1.1293 - accuracy: 0.3333
Epoch 4/15
1/1 - 1s - loss: 0.9910 - accuracy: 1.0000
Epoch 5/15
1/1 - 1s - loss: 0.9626 - accuracy: 0.3333
Epoch 6/15
1/1 - 1s - loss: 0.8901 - accuracy: 0.6667
Epoch 7/15
1/1 - 1s - loss: 0.8179 - accuracy: 1.0000
Epoch 8/15
1/1 - 1s - loss: 0.7582 - accuracy: 1.0000
Epoch 9/15
1/1 - 1s - loss: 0.7029 - accuracy: 1.0000
Epoch 10/15
1/1 - 1s - loss: 0.6512 - accuracy: 1.0000
Epoch 11/15
1/1 - 1s - loss: 0.6015 - accuracy: 1.0000
Epoch 12/15
1/1 - 1s - loss: 0.5439 - accuracy: 1.0000
Epoch 13/15
1/1 - 1s - loss: 0.4830 - accuracy: 1.0000
Epoch 14/15
1/1 - 1s - loss: 0.4385 - accuracy: 1.0000
Epoch 15/15
1/1 - 1s - loss: 0.3991 - accuracy: 1.0000
